In [77]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [78]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [79]:
trailing_num_days = 7

In [80]:
#get all apps > 10 m tvl
all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
res = res[res['tvl'] > 10_000_000] ##greater than 10mil
print(len(res))
# print(res.columns)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.llama.fi:443
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocols HTTP/1.1" 200 1594244


355


In [81]:

protocols = res[['slug','chainTvls']]
re = res['chainTvls']
# r[1].keys()
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
protocols

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_65434/1800157470.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,slug,chainTvls
0,makerdao,[Ethereum]
1,polygon-bridge-&-staking,[Polygon]
2,lido,"[Ethereum, Moonriver, Solana, Terra, Moonbeam]"
3,uniswap,"[Optimism, Ethereum, Polygon, Celo, Arbitrum]"
4,curve,"[Optimism, Harmony, Moonbeam, Ethereum, Aurora..."
...,...,...
350,sashimiswap,"[Binance-borrowed, Binance, Ethereum, Heco, Et..."
351,aavegotchi,"[pool2, Polygon-pool2, Ethereum, Polygon, Poly..."
352,idex,"[Ethereum, Polygon]"
353,guard(helmet),"[Binance, Polygon]"


In [82]:
# protocols = protocols[ protocols['slug'] == 'uniswap-v3' ]
# api_str = 'https://api.llama.fi/protocol/'
# ad = pd.DataFrame( r.get(api_str).json()['chainTvls'] ).T[['tokens']]
# ad

In [83]:
# api_str = 'https://api.llama.fi/protocol/uniswap'
# prot_req = r.get(api_str, headers=header).json()['chainTvls']
# # prot_req['Ethereum']
# prot_req

In [84]:
#get by app
api_str = 'https://api.llama.fi/protocol/'
# print(protocols)
prod = []
for index,proto in protocols.iterrows():
#     print(proto)
    prot = proto['slug']
    chains = proto['chainTvls']
    apic = api_str + prot
#     time.sleep(0.1)
    try:
        prot_req = s.get(apic, headers=header).json()['chainTvls']
#         print(apic)
        for ch in chains:
            ad = pd.json_normalize( prot_req[ch]['tokens'] )
            ad_usd = pd.json_normalize( prot_req[ch]['tokensInUsd'] )
#             ad = ad.merge(how='left')
            if not ad.empty:
                ad = pd.melt(ad,id_vars = ['date'])
                ad = ad.rename(columns={'variable':'token','value':'token_value'})
                ad_usd = pd.melt(ad_usd,id_vars = ['date'])
                ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
                ad = ad.merge(ad_usd,on=['date','token'])
                
                ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
                ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
                ad['protocol'] = prot
                ad['chain'] = ch
        #         ad['start_date'] = pd.to_datetime(prot[1])
                ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
                prod.append(ad)
    except:
        print('err')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.llama.fi:443
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/makerdao HTTP/1.1" 200 1003982
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/polygon-bridge-&-staking HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): defillama-datasets.s3.eu-central-1.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://defillama-datasets.s3.eu-central-1.amazonaws.com:443 "GET /temp/protocol-Polygon%20Bridge%20&%20Staking.json HTTP/1.1" 200 14069079
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/lido HTTP/1.1" 200 443352
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/uniswap HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:https://defillama-datasets.s3.eu-central-1.amazonaws.com:443 "GET /temp/protocol-Uniswap.json HTTP/1.1" 200 13728133
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/curve HTTP/1.1" 200 

err


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/hbtc HTTP/1.1" 200 87549
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/sunswap HTTP/1.1" 200 238489
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/sushiswap HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:Resetting dropped connection: defillama-datasets.s3.eu-central-1.amazonaws.com
DEBUG:urllib3.connectionpool:https://defillama-datasets.s3.eu-central-1.amazonaws.com:443 "GET /temp/protocol-SushiSwap.json HTTP/1.1" 200 80035667
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/venus HTTP/1.1" 200 2354274
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/vvs-finance HTTP/1.1" 200 103838
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/synthetix HTTP/1.1" 200 366868
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/yearn-finance HTTP/1.1" 200 316941
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 

err


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/solend HTTP/1.1" 200 1431327
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/benqi-lending HTTP/1.1" 200 792419
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/armor HTTP/1.1" 200 471639
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/parallel-liquid-crowdloan HTTP/1.1" 502 36
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/tectonic HTTP/1.1" 200 325347
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/nexus-mutual HTTP/1.1" 200 396010


...

DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/dfyn-network HTTP/1.1" 200 4656380
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/bella-protocol HTTP/1.1" 200 155539
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/deeplock HTTP/1.1" 200 68315
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/omm HTTP/1.1" 200 34580
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/thetacash HTTP/1.1" 200 63645
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/nereus-finance HTTP/1.1" 200 118381
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/defrost HTTP/1.1" 200 153831
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/pstake HTTP/1.1" 200 185662
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/karura-liquid-staking HTTP/1.1" 200 77523
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/izumi-finance HTTP/1.

err


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/solv-protocol HTTP/1.1" 200 413779
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/port-finance HTTP/1.1" 200 292159
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/concave HTTP/1.1" 200 87285
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/gro HTTP/1.1" 200 473898
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/risk-harbor HTTP/1.1" 200 122008
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/gains-network HTTP/1.1" 200 187079
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/crystalvale HTTP/1.1" 200 96088
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/proton-loan HTTP/1.1" 200 146053
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/defi-kingdoms HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:Resetting dropped connection: defillama-datasets.s3.eu-central-1.

err


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/acala-swap HTTP/1.1" 200 16630
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/ouchi-finance HTTP/1.1" 200 52881
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/position-exchange HTTP/1.1" 200 289043
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/zookeeper HTTP/1.1" 200 59123
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/88mph HTTP/1.1" 200 1175088
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/sashimiswap HTTP/1.1" 200 1387195
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/aavegotchi HTTP/1.1" 200 661337
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/idex HTTP/1.1" 200 2261810
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/guard(helmet) HTTP/1.1" 200 842461
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/truefi HTTP/1.1" 200

In [85]:
df_df_all = pd.concat(prod)
# df_df_all
print("done api")

done api


In [86]:
# df_df[df_df['protocol']=='perpetual-protocol']
#filter down a bit so we can do trailing comp w/o doing every row
df_df = df_df_all[df_df_all['date'].dt.date >= date.today()-timedelta(days=trailing_num_days +2) ]
# display(df_df['date'].drop_duplicates())
#trailing comp
df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
#now actually filter
df_df = df_df[df_df['date'].dt.date >= date.today()-timedelta(days=trailing_num_days +1) ]
# display(df_df['date'].drop_duplicates())
# df_df[(df_df['protocol'] == 'uniswap') & (df_df['token'] != 'USDT') & (df_df['token_value'] > 0)]

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_65434/3395394319.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [87]:
# # DISTINCT TOKENS

# token_list = df_df.groupby(['token']).count()
# token_list = token_list[token_list['token_value']>=1000]
# token_list.sort_values(by='token_value',inplace=True, ascending=False)
# token_list.reset_index(inplace=True)
# missing_token_list = token_list[~token_list['token'].isin(cg_token_list)]
# missing_token_list
# missing_token_list

In [88]:
# data_df = df_df.merge(cg_df, on=['date','token'],how='inner')
data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

# print(len(data_df))
data_df.sort_values(by='date',inplace=True)
# print(len(data_df))
# data_df['net_token_flow'] = data_df.groupby(['token','protocol','chain'])['token_value'].apply(lambda x: x- x.shift(1))
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']

# print(len(data_df))
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


# data_df = data_df[data_df['token'] != 'RVRS'] #Harmony error
# data_df = data_df[data_df['token'] != 'usn'] #Harmony error
# data_df = data_df[data_df['protocol'] != 'shibaswap'] #Harmony error
data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar
data_df = data_df[~data_df['net_dollar_flow'].isna()] #50 bil error bar
# display(data_df[
# #         (data_df['protocol'] == 'sushiswap') & (data_df['chain'] == 'Harmony')
#         (data_df['net_dollar_flow'].notna()) & (data_df['net_dollar_flow'] != 0)
#         ].sort_values('net_dollar_flow'))


# data_df.sort_values('net_dollar_flow')


In [89]:
# data_df[['date']].drop_duplicates().sort_values('date')
# data_df[data_df['date'].dt.date != data_df['date']].sort_values('date')

In [90]:
# data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')

# segment_cols = ['date','protocol','chain','token']

In [91]:
netdf_df = data_df[['date','protocol','chain','net_dollar_flow','usd_value']]
netdf_df = netdf_df.fillna(0)
netdf_df = netdf_df.groupby(['date','protocol','chain']).sum(['net_dollar_flow','usd_value']) ##agg by app
# display(netdf_df)
#usd_value is the TVL on a given day
netdf_df = netdf_df.groupby(['date','protocol','chain','usd_value']).sum(['net_dollar_flow'])
netdf_df.reset_index(inplace=True)
# display(netdf_df)
netdf_df['cumul_net_dollar_flow'] = netdf_df[['protocol','chain','net_dollar_flow']]\
                                    .groupby(['protocol','chain']).cumsum()
netdf_df.reset_index(inplace=True)
netdf_df.drop(columns=['index'],inplace=True)


In [92]:
#get latest
netdf_df['rank_desc'] = netdf_df.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
# netdf_df[netdf_df['chain'] == 'Klaytn']

In [98]:
summary_df = netdf_df[  ( netdf_df['rank_desc'] == 1 ) &\
                        (~netdf_df['chain'].str.contains('-borrowed')) &\
                        (~netdf_df['chain'].str.contains('-staking')) &\
                        (~netdf_df['chain'].str.contains('-pool2')) &\
                        (~( netdf_df['chain'] == 'treasury') )
                        ]
summary_df = summary_df.sort_values(by='cumul_net_dollar_flow',ascending=False)
summary_df['pct_of_tvl'] = 100* summary_df['net_dollar_flow'] / summary_df['usd_value']
summary_df['flow_direction'] = np.where(summary_df['cumul_net_dollar_flow']>=0,1,-1)
summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])
# display(summary_df)
fig = px.treemap(summary_df[summary_df['abs_cumul_net_dollar_flow'] !=0], \
                 path=[px.Constant("all"), 'chain', 'protocol'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values='abs_cumul_net_dollar_flow', color='flow_direction'
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='Spectral'
                )
# fig.data[0].textinfo = 'label+text+value'
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.update_layout(tickprefix = '$')

# fig.show()
fig.write_image(prepend + "img_outputs/svg/net_app_flows.svg") #prepend + 
fig.write_image(prepend + "img_outputs/png/net_app_flows.png") #prepend + 
fig.write_html(prepend + "img_outputs/net_app_flows.html", include_plotlyjs='cdn')

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [99]:
# netdf_df_plot = netdf_df[netdf_df['chain']=='Ethereum']

# fig = px.line(netdf_df_plot, x="date", y="net_dollar_flow", color="protocol", \
#              title="Daily Net Dollar Flow since Program Announcement",\
#             labels={
#                      "date": "Day",
#                      "net_dollar_flow": "Net Dollar Flow (N$F)"
#                  }
#             )
# fig.update_layout(
#     legend_title="App Name"
# )
# fig.update_layout(yaxis_tickprefix = '$')
# fig.show()

# # cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
# #              title="Cumulative Dollar Flow since Program Announcement",\
# #                    labels={
# #                      "date": "Day",
# #                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
# #                  }
# #             ,areamode='group')
# # cumul_fig.update_layout(yaxis_tickprefix = '$')
# # cumul_fig.show()


# cumul_fig = go.Figure()
# proto_names = netdf_df_plot['protocol'].drop_duplicates()
# print(proto_names)
# for p in proto_names:
#     cumul_fig.add_trace(go.Scatter(x=netdf_df_plot[netdf_df_plot['protocol'] == p]['date'] \
#                                    , y=netdf_df_plot[netdf_df_plot['protocol'] == p]['cumul_net_dollar_flow'] \
#                                     ,name = p\
#                                   ,fill='tozeroy')) # fill down to xaxis

# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.update_layout(
#     title="Cumulative Dollar Flow since Program Announcement",
#     xaxis_title="Day",
#     yaxis_title="Cumulative Net Dollar Flow (N$F)",
#     legend_title="App Name",
# #     color_discrete_map=px.colors.qualitative.G10
# )
# cumul_fig.show()

In [101]:
# ! jupyter nbconvert --to python total_app_net_flows.ipynb

[NbConvertApp] Converting notebook total_app_net_flows.ipynb to python
[NbConvertApp] Writing 10005 bytes to total_app_net_flows.py
